# Object detection with Tensorflow object detection API

## setting GPU

In [ ]:
# ONLY IF YOU HAVE SOME MANY RESTART SESSION
%reset -f
import os
os.kill(os.getpid(), 9)


In [ ]:
# Install core GPU runtime deps TensorFlow 2.15 needs
!apt-get update -y
!apt-get install -y --no-install-recommends \
    cuda-command-line-tools-12-2 \
    cuda-cupti-12-2 \
    libcudnn8=8.9.4.25-1+cuda12.2 \
    libcudnn8-dev=8.9.4.25-1+cuda12.2 \
    libcusparse-dev-12-2

# Clean remnants
!rm -rf /content/sample_data


In [ ]:
# Check GPU (you should see NVIDIA L4 or similar)
!nvidia-smi

## Installation

You'll start by installing the Tensorflow 2 [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection).

In [ ]:
# 2. TensorFlow 2.15 + matching Keras + tf-slim + Protobuf
!pip install --upgrade pip
!pip install tensorflow==2.15.0 keras==2.15.0 tf-slim protobuf==3.20.*

**IMPORTANT**: Please restart the runtime for the changes to take effect. You can either click the `Restart Runtime` button in the output of the cell above, or go to `Runtime > Restart Session` on the Menu bar above.

In [ ]:
# 3. Clone the models repo
%cd /content
!git clone https://github.com/tensorflow/models.git
%cd models/research

In [ ]:
# 4. Compile protobufs
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# 5. Install the Object Detection API properly
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
# Yes install again then restart again
!pip install tensorflow==2.15.0 keras==2.15.0 tf-slim protobuf==3.20.*

**IMPORTANT**: Please restart the runtime for the changes to take effect. You can either click the `Restart Runtime` button in the output of the cell above, or go to `Runtime > Restart Session` on the Menu bar above.

In [ ]:
!cp /content/tfexample_decoder.py /usr/local/lib/python3.11/dist-packages/tf_slim/data/tfexample_decoder.py


In [ ]:
!head -n 20 /usr/local/lib/python3.11/dist-packages/tf_slim/data/tfexample_decoder.py


In [ ]:
# Remove broken estimator import
!sed -i '/from tensorflow.compat.v1 import estimator as tf_estimator/d' object_detection/inputs.py
!sed -i '/import tensorflow_estimator as tf_estimator/d' object_detection/inputs.py


## Imports

Let's now import the packages you will use in this assignment.

In [ ]:
import sys
print("🧠 Python version:", sys.version)


In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util, config_util
from object_detection import model_lib_v2

print("✅ TensorFlow version:", tf.__version__)
print("✅ GPU devices:", tf.config.list_physical_devices('GPU'))
print("✅ Object Detection API loaded successfully!")


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import zipfile
import io
import scipy.misc
import numpy as np

import glob
import imageio
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

## Roboflow data

In [ ]:
# Install and set up Roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="3zX0IiVPf7UfB1zBmrKv")

In [ ]:
# Load dataset
workspace = rf.workspace("ssdcctv")
project = workspace.project("my-first-project-nwbay")
dataset = project.version(1).download("tfrecord")

In [ ]:
# Define dataset paths
dataset_location = dataset.location
train_data_path = os.path.join(dataset_location, "train", "objects.tfrecord")
val_data_path = os.path.join(dataset_location, "valid", "objects.tfrecord")
label_map_path = os.path.join(dataset_location, "train", "objects_label_map.pbtxt")
test_data_path = os.path.join(dataset_location, "test", "objects.tfrecord")

In [ ]:
# Load label map
categories = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)
num_classes = len(categories)

## pipeline configuration

In [ ]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'efficientdet-d0'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    'ssd_resnet50_v1': {
        'model_name': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz',
    }
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
# Set training parameters for the model
num_steps = 40000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

In [ ]:
# Set file locations
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /content/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_data_path), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_data_path), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_path), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

In [ ]:
# (Optional) Display the custom configuration file's contents
!cat /content/models/mymodel/pipeline_file.config

In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/models/mymodel/pipeline_file.config'
model_dir = '/content/training/'

## Training

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/training/train'


In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path="/content/models/mymodel/pipeline_file.config" \
  --model_dir="/content/training/" \
  --alsologtostderr \
  --num_train_steps=40000 \
  --sample_1_of_n_eval_examples=1


## Export model inference graph

In [ ]:

# === Export the model ===
!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path="/content/models/mymodel/pipeline_file.config" \
    --trained_checkpoint_dir="/content/training" \
    --output_directory="/content/models/my_model/exported_model"


In [ ]:
import shutil

# Zip the exported model
export_dir = '/content/models/my_model/exported_model'
zip_path = '/content/models/my_model/exported_model.zip'

shutil.make_archive(zip_path.replace('.zip', ''), 'zip', export_dir)


In [ ]:
from google.colab import files

# Download the .zip file
files.download(zip_path)


## Test Trained model with test image

In [ ]:
import six
import time
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
# Load the exported SavedModel
model_dir = "/content/models/my_model/exported_model/saved_model"
detect_fn = tf.saved_model.load(model_dir)

In [ ]:
#not used?
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data)).convert("RGB")
    return np.array(image)


In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

In [ ]:
#not used?
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

In [ ]:
# Helper to load and display image with predictions
def show_inference(model, image_path):
    image_np = image_tensor.numpy() if tf.is_tensor(image_tensor) else image_tensor
    output_dict = run_inference_for_single_image(model, image_np)

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=10,
        min_score_thresh=0.3,
        agnostic_mode=False)

    plt.figure(figsize=(12,8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()

In [ ]:
# Run inference on test images
test_images_folder = os.path.join(dataset_location, "test")
image_files = [os.path.join(test_images_folder, f) for f in os.listdir(test_images_folder) if f.endswith(".jpg") or f.endswith(".png")]

In [ ]:
# Show predictions for a few test images
for image_path in image_files[:5]:
    print(f"Running inference on: {image_path}")
    show_inference(detect_fn, image_path)

In [ ]:
def parse_tfrecord(example):
    features = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }
    return tf.io.parse_single_example(example, features)

In [ ]:
from PIL import ImageDraw

def run_model_on_roboflow_tfrecord(tfrecord_path, model, num_images=5):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(parse_tfrecord)

    for i, record in enumerate(dataset.take(num_images)):
        image_bytes = record['image/encoded'].numpy()
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        image_np = np.array(image)

        # Run inference
        output_dict = run_inference_for_single_image(model, image_np)

        # Visualization
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=0.3,
            agnostic_mode=False)

        plt.figure(figsize=(8, 8))
        plt.imshow(image_np)
        plt.axis('off')
        plt.title(record['image/filename'].numpy().decode('utf-8'))
        plt.show()

# Run it on test TFRecord
run_model_on_roboflow_tfrecord(test_data_path, detect_fn, num_images=5)


In [ ]:
def run_inference_and_show(image_path):
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

    detections = detect_fn(input_tensor)

    # Process results
    num_detections = int(detections.pop('num_detections'))
    detections = {k: v[0, :num_detections].numpy()
                  for k, v in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # Visualize results
    image_with_detections = image_np.copy()
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=20,
        min_score_thresh=0.4,
        agnostic_mode=False)

    plt.figure(figsize=(10, 10))
    plt.imshow(image_with_detections)
    plt.axis('off')
    plt.show()

In [ ]:
import os
import tensorflow as tf
from PIL import Image
import io

# Set TFRecord and output path
tfrecord_path = '/content/My-First-Project-1/test/objects.tfrecord'
output_dir = '/content/extracted_test_images'
os.makedirs(output_dir, exist_ok=True)

# Define the TFRecord schema
def parse_tfrecord2(example):
    features = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
    }
    return tf.io.parse_single_example(example, features)

# Read and extract images
raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
parsed_dataset = raw_dataset.map(parse_tfrecord2)

for i, parsed_record in enumerate(parsed_dataset):
    img_bytes = parsed_record['image/encoded'].numpy()
    filename = parsed_record['image/filename'].numpy().decode('utf-8')
    image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    image.save(os.path.join(output_dir, filename))

    if i >= 10:  # Save only 5 images for now
        break

print("✅ Saved extracted images to:", output_dir)


In [ ]:
# Replace with one of the extracted images
image_path = '/content/extracted_test_images/25994-colt-fe-jual-truk-engkel-fe-71_jpg.rf.0204a33669487196240645aa25d180e6.jpg'
run_inference_and_show(image_path)


In [ ]:
import shutil

# Zip the folder
shutil.make_archive('/content/extracted_test_images', 'zip', '/content/extracted_test_images')


In [ ]:
from google.colab import files
files.download('/content/extracted_test_images.zip')


## Evaluation MAP

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path="/content/models/mymodel/pipeline_file.config" \
  --model_dir="/content/training/" \
  --checkpoint_dir="/content/training/" \
  --run_once=True


